In [1]:
from market_imbalance import MarketImbalance, sort_imbalances_by_timestamp
from plot_market_candles_with_plotly import plot_market_candles_with_plotly

def find_imbalances_after_fall(historical_candles):
    imbalances_after_fall = []
    for index in range(len(historical_candles)):
        if index == 0 or index == len(historical_candles) - 1:
            continue
        prev_candle = historical_candles[index - 1]
        current_candle = historical_candles[index]
        next_candle = historical_candles[index + 1]
        if prev_candle.low_price > next_candle.high_price:
            delta_to_be_filled_in = prev_candle.low_price - next_candle.high_price
            
            imbalance = MarketImbalance(
                imbalance_type="imbalance_after_fall",
                timestamp=current_candle.timestamp,
                open_price=next_candle.high_price,
                close_price=prev_candle.low_price,
                delta_to_be_filled_in=delta_to_be_filled_in,
                is_full_filled=False,
                was_fullfilled_at=None,
                time_to_be_fullfilled=None,
                is_partially_filled=False,
                remaining_delta_open_price=None,
                remaining_delta_to_be_filled_in=None,
                candles_of_identification=(prev_candle, current_candle, next_candle),
                candles_of_fullfilling=None,
                candles_of_partfilling=None
            )
            imbalances_after_fall.append(imbalance)
    return imbalances_after_fall

def find_imbalances_after_rise(historical_candles):
    imbalances_after_rise = []
    for index in range(len(historical_candles)):
        if index == 0 or index == len(historical_candles) - 1:
            continue
        prev_candle = historical_candles[index - 1]
        current_candle = historical_candles[index]
        next_candle = historical_candles[index + 1]
        if prev_candle.high_price < next_candle.low_price:
            delta_to_be_filled_in = next_candle.low_price - prev_candle.high_price
            
            imbalance = MarketImbalance(
                imbalance_type="imbalance_after_rise",
                timestamp=current_candle.timestamp,
                open_price=next_candle.low_price,
                close_price=prev_candle.high_price,
                delta_to_be_filled_in=delta_to_be_filled_in,
                is_full_filled=False,
                was_fullfilled_at=None,
                time_to_be_fullfilled=None,
                is_partially_filled=False,
                remaining_delta_open_price=None,
                remaining_delta_to_be_filled_in=None,
                candles_of_identification=(prev_candle, current_candle, next_candle),
                candles_of_fullfilling=None,
                candles_of_partfilling=None
            )
            imbalances_after_rise.append(imbalance)
    return imbalances_after_rise

def get_unfilled_imbalances(imbalances):
    unfilled_imbalances = [imbalance for imbalance in imbalances if not imbalance.is_full_filled]
    return unfilled_imbalances

def get_fullfilled_imbalances(imbalances):
    fullfilled_imbalances = [imbalance for imbalance in imbalances if imbalance.is_full_filled]
    return fullfilled_imbalances


In [4]:
# !pip install python-binance
import os
from interact_with_binance import fetch_ohlcv, fetch_ohlcv_as_df
from market_candles import MarketCandle
from binance.client import Client
from convert_ts_to_datetime import convert_ts_to_datetime


api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')
client = Client(api_key, api_secret)

# Récupère les données de marché en *temps différé* sur Binance
interval_value = Client.KLINE_INTERVAL_1HOUR
from_date = "15 avril 2024"

# Permet d'afficher les données à l'heure française

historical_candles = fetch_ohlcv(client, symbol = "BTCUSDT", interval=interval_value, from_date=from_date) #"1 Jan, 2015"
historical_candles_df = fetch_ohlcv_as_df(client, symbol = "BTCUSDT", interval=interval_value, from_date=from_date) #"1 Jan, 2015"
plot_market_candles_with_plotly(historical_candles_df)


In [8]:
# Récupère les imbalances à partir des infos de marché
imbalances_after_fall = find_imbalances_after_fall(historical_candles)
imbalances_after_rise = find_imbalances_after_rise(historical_candles)

for imb in imbalances_after_fall:
    print(imb.imbalance_type, convert_ts_to_datetime(imb.timestamp))


imbalance_after_fall 2024-04-15 13:00:00+02:00
imbalance_after_fall 2024-04-15 16:00:00+02:00
imbalance_after_fall 2024-04-16 12:00:00+02:00


In [ ]:
# Identifie uniquement les imbalances déjà comblés
fullfilled_imbalances_after_fall = get_fullfilled_imbalances(imbalances_after_fall)
fullfilled_imbalances_after_rise = get_fullfilled_imbalances(imbalances_after_rise)

# Identifie uniquement les imbalances qu'il reste à combler
unfilled_imbalances_after_fall = get_unfilled_imbalances(imbalances_after_fall)
unfilled_imbalances_after_rise = get_unfilled_imbalances(imbalances_after_rise)

# Percentage of gap filled after fall
percentage_filled_after_fall = (len(fullfilled_imbalances_after_fall) / len(imbalances_after_fall)) * 100
print(f'% of filled gap after fall = {percentage_filled_after_fall} with {len(fullfilled_imbalances_after_fall)} fullfilled imbalances on {len(imbalances_after_fall)} in total')

# Percentage of gap filled after rise
percentage_filled_after_rise = (len(fullfilled_imbalances_after_rise) / len(imbalances_after_rise)) * 100
print(f'% of filled gap after rise = {percentage_filled_after_rise} with {len(fullfilled_imbalances_after_rise)} fullfilled imbalances on {len(imbalances_after_rise)} in total')



In [ ]:

for imb in unfilled_imbalances_after_rise[-10:] : 
    print('fullfilled =',imb.is_full_filled, ',imbalance that started on', convert_ts_to_datetime(imb.timestamp), 'at an open price =', imb.open_price, 'at on close_price =', imb.close_price, 'with a delta =', round(imb.delta_to_be_filled_in, 2))

In [ ]:
# Affiche les imbalances non comblés sur le graphique
unfilled_imbalances = []
unfilled_imbalances.extend(unfilled_imbalances_after_fall)
unfilled_imbalances.extend(unfilled_imbalances_after_rise)

sorted_imbalances = sort_imbalances_by_timestamp(unfilled_imbalances)
sorted_imbalances[-10:]

for imb in sorted_imbalances[-10:]:
    print(imb.timestamp, imb.open_price, imb.close_price, round(imb.delta_to_be_filled_in, 2))



In [ ]:
# Percentage of gap filled after fall
percentage_filled_after_fall = (len(fullfilled_imbalances_after_fall) / len(find_imbalances_after_fall)) * 100
print(f'% of filled gap after fall = {percentage_filled_after_fall} with {len(fullfilled_imbalances_after_fall)} fullfilled imbalances on {len(imbalances_after_fall)} in total')

# Percentage of gap filled after rise
percentage_filled_after_rise = (len(fullfilled_imbalances_after_rise) / len(find_imbalances_after_rise)) * 100
print(f'% of filled gap after rise = {percentage_filled_after_rise} with {len(fullfilled_imbalances_after_rise)} fullfilled imbalances on {len(imbalances_after_rise)} in total')

# Percentage of gap filled in both directions
total_imbalances = (len(fullfilled_imbalances_after_fall) + len(fullfilled_imbalances_after_rise))
percentage_filled = (len(fullfilled_imbalances) / len(total_imbalances)) * 100
print(f'% of filled gap = {percentage_filled} with {len(total_imbalances)} fullfilled imbalances on {len(fullfilled_imbalances)} in total')
